In [1]:
import socket
import sys
from collections import namedtuple
from struct import unpack
import asyncore
import rospy
from std_msgs.msg import Float32
from std_msgs.msg import String

In [ ]:
#received_message=sock.recv(50)
#print(received_message)

In [ ]:
    with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as sock:
        ip="192.168.1.120"
        port=8000
    # Connect the socket to the port where the server is listening
        server_address = (ip,port)
        print( 'connecting to %s port %s' % server_address)
        somesocket=sock.connect(server_address)
        sock.sendall("testing".encode())
        received_message=sock.recv(50)



connecting to 192.168.1.120 port 8000


In [ ]:
print(len(received_message))

In [ ]:
touple_from_tcp=unpack('ddddddcc', received_message)
touple_from_tcp

In [ ]:
Message = namedtuple('Message', ['position1','speed1','torque1','position2','speed2','torque2','first_letter','second_letter'])
my_message=Message(*touple_from_tcp)
list(my_message)

In [ ]:
my_message.position1

In [ ]:
rospy.init_node('talker', anonymous=True)


Message = namedtuple('Message', ['position1','speed1','torque1','position2','speed2','torque2','first_letter','second_letter'])


class HTTPClient(asyncore.dispatcher):

    def __init__(self, host="192.168.1.120", port=8000, message_size=50):
        self.pub0 = rospy.Publisher('/rail1/position', Float32, queue_size=10)
        self.pub1 = rospy.Publisher('/rail1/speed', Float32, queue_size=10)
        self.pub2 = rospy.Publisher('/rail1/torque', Float32, queue_size=10)
        self.pub3 = rospy.Publisher('/rail2/position', Float32, queue_size=10)
        self.pub4 = rospy.Publisher('/rail2/speed', Float32, queue_size=10)
        self.pub5 = rospy.Publisher('/rail2/torque', Float32, queue_size=10)
        asyncore.dispatcher.__init__(self)
        self.create_socket()
        self.message_size=message_size
        self.connect( (host, port) )
        self.buffer = bytes('testing','utf8')
        self.counter=0
        rospy.Subscriber("chatter", String, self.callback)
        self.last_message=""
    def handle_connect(self):
        print("connected")
        pass
    def callback(self,message):
        print(message) #String from "chatter" Publisher
        self.buffer =bytes(message.data,'utf8')
        self.handle_write()
    def handle_close(self):
        self.close()
    def decode_message(self,message):
        if(len(message)==self.message_size):
            touple_from_tcp=unpack('ddddddcc', message)
            my_message=Message(*touple_from_tcp)
            return my_message
        return False
    def handle_read(self):
        byte_message=self.recv(self.message_size)
        
        self.last_message=self.decode_message(byte_message)
        #print(self.last_message)
        if(self.last_message):
            try:
                #print(self.last_message.position1)
                self.pub0.publish(self.last_message.position1)
                self.pub1.publish(self.last_message.speed1)
                self.pub2.publish(self.last_message.torque1)
                self.pub3.publish(self.last_message.position2)
                self.pub4.publish(self.last_message.speed2)
                self.pub5.publish(self.last_message.torque2)
            except:
                print(":-(")
        self.counter+=1
        
    def writable(self):
        return (len(self.buffer) > 0)

    def handle_write(self):
        sent = self.send(self.buffer)
        #self.buffer = self.buffer[sent:]




In [ ]:
client = HTTPClient('192.168.1.120',8000)
asyncore.loop()

In [ ]:
client.socket.close()

In [ ]:
client.counter